# CalcSet Grouping and Loop Detection Demo
This notebook demonstrates live testing of grouping and loop detection using a real-world resource calculation CalcSet. We'll build a multi-step workflow with variables, calculations, and domains, then decompile it to inspect the generated Python snippet.

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# Example: Build a resource CalcSet with external variables, multiple domains, and calculations
from pollywog.core import CalcSet, Number, Category, Filter, If, IfRow
import pollywog as pw
pw.set_theme("dark")
from pollywog.conversion.decomp import decompile_calcset

# External variables (not part of CalcSet)
# Au, Ag, Cu, Zn, tonnage, density, recovery, grade_class
variables = ["Au", "Ag", "Cu", "Zn"]
domains = ["HighAuHighCu", "HighAuLowCu", "LowAuHighCu", "LowAuLowCu"]
weathering = ["OXI", "TRANS", "FRESH"]

# Step 1: Domain assignment (based on grade and other variables)
items = [
    Category("domain", [If(
        rows=[
            IfRow(["[Au] > 2 and [Cu] > 1"], ["HighAuHighCu"]),
            IfRow(["[Au] > 2 and [Cu] <= 1"], ["HighAuLowCu"]),
            IfRow(["[Au] <= 2 and [Cu] > 1"], ["LowAuHighCu"]),
            IfRow(["[Au] <= 2 and [Cu] <= 1"], ["LowAuLowCu"]),
        ],
        otherwise=["Undefined"]
    )])
]

# Step 2: Calculations per domain and input variable combinations\
# for weath in weathering:
#     for dom in domains:
#         for metal in variables:
#             items.append(Number(
#                 f"{metal}_{dom}_{weath}",
#                 [If(
#                     rows=[IfRow([f"[domain] == '{dom}' and [weathering] == '{weath}'"], [f"[{metal}]"])],
#                     otherwise=[0]
#                 )]
#             ))
for dom in domains:
    for metal in variables:
        items.append(Number(
            f"{metal}_{dom}",
            [If(
                rows=[IfRow([f"[domain] == '{dom}'"], [f"[{metal}]"])],
                otherwise=[0]
            )]
        ))


# Step 3: Resource calculation per domain
for dom in domains:
    for metal in variables:
        items.append(Number(f"resource_{metal}_{dom}", [f"[{metal}_{dom}] * [tonnage_{dom}] * [recovery_{dom}] * [density_{dom}]"]))

calcset = CalcSet(items)
calcset
# calcset.topological_sort()

{
  "type": "calculation-set",
  "items": [
    {
      "type": "calculation",
      "name": "domain",
      "equation": {
        "type": "equation",
        "comment": "",
        "statement": {
          "type": "list",
          "children": [
            "",
            {
              "type": "if",
              "rows": [
                {
                  "type": "if_row",
                  "test": {
                    "type": "list",
                    "children": [
                      "[Au] > 2 and [Cu] > 1"
                    ]
                  },
                  "result": {
                    "type": "list",
                    "children": [
                      "HighAuHighCu"
                    ]
                  }
                },
                {
                  "type": "if_row",
                  "test": {
                    "type": "list",
                    "children": [
                      "[Au] > 2 and [Cu] <= 1"
                    ]
                  },
                  "result": {
                    "type": "list",
                    "children": [
                      "HighAuLowCu"
                    ]
                  }
                },
                {
                  "type": "if_row",
                  "test": {
                    "type": "list",
                    "children": [
                      "[Au] <= 2 and [Cu] > 1"
                    ]
                  },
                  "result": {
                    "type": "list",
                    "children": [
                      "LowAuHighCu"
                    ]
                  }
                },
                {
                  "type": "if_row",
                  "test": {
                    "type": "list",
                    "children": [
                      "[Au] <= 2 and [Cu] <= 1"
                    ]
                  },
                  "result": {
                    "type": "list",
                    "children": [
                      "LowAuLowCu"
                    ]
                  }
                }
              ],
              "otherwise": {
                "type": "list",
                "children": [
                  "Undefined"
                ]
              }
            },
            ""
          ]
        }
      },
      "comment": "",
      "calculation_type": "string"
    },
    {
      "type": "calculation",
      "name": "Au_HighAuHighCu",
      "equation": {
        "type": "equation",
        "comment": "",
        "statement": {
          "type": "list",
          "children": [
            "",
            {
              "type": "if",
              "rows": [
                {
                  "type": "if_row",
                  "test": {
                    "type": "list",
                    "children": [
                      "[domain] == 'HighAuHighCu'"
                    ]
                  },
                  "result": {
                    "type": "list",
                    "children": [
                      "[Au]"
                    ]
                  }
                }
              ],
              "otherwise": {
                "type": "list",
                "children": [
                  "",
                  0,
                  ""
                ]
              }
            },
            ""
          ]
        }
      },
      "comment": "",
      "calculation_type": "number"
    },
    {
      "type": "calculation",
      "name": "Ag_HighAuHighCu",
      "equation": {
        "type": "equation",
        "comment": "",
        "statement": {
          "type": "list",
          "children": [
            "",
            {
              "type": "if",
              "rows": [
                {
                  "type": "if_row",
                  "test": {
                    "type": "list",
                    "children": [
                      "[domain] == 'HighAuHighCu'"
                    ]
                  },
                  "re

In [2]:
# Decompile the CalcSet and inspect the generated Python snippet
snippet = decompile_calcset(calcset)
print(snippet)

from pollywog.core import CalcSet, Number, Category, Variable, Filter, If, IfRow
items = []
items.append(Category(name='domain', expression=[If(rows=[IfRow(condition=['[Au] > 2 and [Cu] > 1'], value=['HighAuHighCu']), IfRow(condition=['[Au] > 2 and [Cu] <= 1'], value=['HighAuLowCu']), IfRow(condition=['[Au] <= 2 and [Cu] > 1'], value=['LowAuHighCu']), IfRow(condition=['[Au] <= 2 and [Cu] <= 1'], value=['LowAuLowCu'])], otherwise=['Undefined'])]))
for number_var in ['Ag_HighAuHigh', 'Ag_HighAuLowC', 'Ag_LowAuHighC', 'Ag_LowAuLowCu', 'Au_HighAuHigh', 'Au_HighAuLowC', 'Au_LowAuHighC', 'Au_LowAuLowCu', 'Cu_HighAuHigh', 'Cu_HighAuLowC', 'Cu_LowAuHighC', 'Cu_LowAuLowCu', 'Zn_HighAuHigh', 'Zn_HighAuLowC', 'Zn_LowAuHighC', 'Zn_LowAuLowCu']:
    items.append(Number(name=f'{number_var}Cu', expression=[If(rows=[IfRow(condition=["[domain] == 'HighAuHighCu'"], value=['[Au]'])], otherwise=[0])]))
    items.append(Number(name=f'{number_var}Cu', expression=[If(rows=[IfRow(condition=["[domain] == 'High

In [4]:
# Execute the generated snippet to reconstruct the CalcSet
namespace = {}
exec(snippet, namespace)
reconstructed = namespace["calcset"]
# assert [item.to_dict() for item in calcset.items] == [item.to_dict() for item in reconstructed.items]
reconstructed

TypeError: Object of type If is not JSON serializable

Calculations domain string if [Au] > 2 and [Cu] > 1 →      HighAuHighCu if [Au] > 2 and [Cu] <= 1 →      HighAuLowCu if [Au] <= 2 and [Cu] > 1 →      LowAuHighCu if [Au] <= 2 and [Cu] <= 1 →      LowAuLowCu otherwise →      Undefined Ag_HighAuHighCu number <pollywog.core.If object at 0x000002207FE10AD0> Ag_HighAuLowCu number <pollywog.core.If object at 0x000002207FE11710> Ag_LowAuHighCu number <pollywog.core.If object at 0x000002207FE10090> Ag_LowAuLowCu number <pollywog.core.If object at 0x000002207FE11810> Au_HighAuHighCu number <pollywog.core.If object at 0x000002207FE126D0> Au_HighAuLowCu number <pollywog.core.If object at 0x000002207FE108D0> Au_LowAuHighCu number <pollywog.core.If object at 0x000002207FE13D90> Au_LowAuLowCu number <pollywog.core.If object at 0x000002207FE13F50> Cu_HighAuHighCu number <pollywog.core.If object at 0x000002207FE13CD0> Cu_HighAuLowCu number <pollywog.core.If object at 0x000002207FE136D0> Cu_LowAuHighCu number <pollywog.core.If object at 0x000002207FE13090> Cu_LowAuLowCu number <pollywog.core.If object at 0x000002207FE13850> Zn_HighAuHighCu number <pollywog.core.If object at 0x000002207FE11690> Zn_HighAuLowCu number <pollywog.core.If object at 0x000002207FE125D0> Zn_LowAuHighCu number <pollywog.core.If object at 0x000002207FE12650> Zn_LowAuLowCu number <pollywog.core.If object at 0x00000220002A2190> resource_Au_HighAuHighCu number [Au_HighAuHighCu] * [tonnage_HighAuHighCu] * [recovery_HighAuHighCu] * [density_HighAuHighCu] resource_Ag_HighAuHighCu number [Ag_HighAuHighCu] * [tonnage_HighAuHighCu] * [recovery_HighAuHighCu] * [density_HighAuHighCu] resource_Cu_HighAuHighCu number [Cu_HighAuHighCu] * [tonnage_HighAuHighCu] * [recovery_HighAuHighCu] * [density_HighAuHighCu] resource_Zn_HighAuHighCu number [Zn_HighAuHighCu] * [tonnage_HighAuHighCu] * [recovery_HighAuHighCu] * [density_HighAuHighCu] resource_Au_HighAuLowCu number [Au_HighAuLowCu] * [tonnage_HighAuLowCu] * [recovery_HighAuLowCu] * [density_HighAuLowCu] resource_Ag_HighAuLowCu number [Ag_HighAuLowCu] * [tonnage_HighAuLowCu] * [recovery_HighAuLowCu] * [density_HighAuLowCu] resource_Cu_HighAuLowCu number [Cu_HighAuLowCu] * [tonnage_HighAuLowCu] * [recovery_HighAuLowCu] * [density_HighAuLowCu] resource_Zn_HighAuLowCu number [Zn_HighAuLowCu] * [tonnage_HighAuLowCu] * [recovery_HighAuLowCu] * [density_HighAuLowCu] resource_Au_LowAuHighCu number [Au_LowAuHighCu] * [tonnage_LowAuHighCu] * [recovery_LowAuHighCu] * [density_LowAuHighCu] resource_Ag_LowAuHighCu number [Ag_LowAuHighCu] * [tonnage_LowAuHighCu] * [recovery_LowAuHighCu] * [density_LowAuHighCu] resource_Cu_LowAuHighCu number [Cu_LowAuHighCu] * [tonnage_LowAuHighCu] * [recovery_LowAuHighCu] * [density_LowAuHighCu] resource_Zn_LowAuHighCu number [Zn_LowAuHighCu] * [tonnage_LowAuHighCu] * [recovery_LowAuHighCu] * [density_LowAuHighCu] resource_Au_LowAuLowCu number [Au_LowAuLowCu] * [tonnage_LowAuLowCu] * [recovery_LowAuLowCu] * [density_LowAuLowCu] resource_Ag_LowAuLowCu number [Ag_LowAuLowCu] * [tonnage_LowAuLowCu] * [recovery_LowAuLowCu] * [density_LowAuLowCu] resource_Cu_LowAuLowCu number [Cu_LowAuLowCu] * [tonnage_LowAuLowCu] * [recovery_LowAuLowCu] * [density_LowAuLowCu] resource_Zn_LowAuLowCu number [Zn_LowAuLowCu] * [tonnage_LowAuLowCu] * [recovery_LowAuLowCu] * [density_LowAuLowCu]